In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from minisom import MiniSom # This is for the unsupervised clustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
def read_data():
    path = r'C:\\Users\\jesse\\Downloads\\clases\\electrical data\\london data' # The path were all the .csv are kept"
    all_files = glob.glob(path + "/*.csv")
    
    li = [] # list for the .csv
    
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, na_values = 0)
        li.append(df) # append all .csv to dataframe
        
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [3]:
df = read_data() # get the data

In [4]:
df

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
0,MAC000002,Std,10/12/2012 0:30,NaN,ACORN-A,Affluent
1,MAC000002,Std,10/12/2012 1:00,NaN,ACORN-A,Affluent
2,MAC000002,Std,10/12/2012 1:30,NaN,ACORN-A,Affluent
3,MAC000002,Std,10/12/2012 2:00,NaN,ACORN-A,Affluent
4,MAC000002,Std,10/12/2012 2:30,NaN,ACORN-A,Affluent
...,...,...,...,...,...,...
167932469,MAC004221,Std,2013-08-06 04:30:00.0000000,0.017,ACORN-E,Affluent
167932470,MAC004221,Std,2013-08-06 05:00:00.0000000,0.017,ACORN-E,Affluent
167932471,MAC004221,Std,2013-08-06 05:30:00.0000000,0.017,ACORN-E,Affluent
167932472,MAC004221,Std,2013-08-06 06:00:00.0000000,0.017,ACORN-E,Affluent


In [5]:
df.rename(columns={'KWH/hh (per half hour) ' : 'KWH/hh'}, inplace=True) # rename column to make easier to read.

In [6]:
# Missing data.
df.fillna(0.0, inplace=True)
df['KWH/hh'].replace('Null', 0, inplace=True) # ValueError: could not convert string to float: 'Null'
df['KWH/hh'] = df['KWH/hh'].astype(float) 

In [7]:
df['DateTime'] = pd.to_datetime(df['DateTime']) # convert to datetime datetype

In [8]:
df.rename(columns={'Acorn_grouped_ACORN-' : 'Acorn_grouped_ACORN'}, inplace=True) # rename these col so I can drop them.
df.rename(columns={'Acorn_grouped_ACORN-U' : 'Acorn_grouped_ACORNU'}, inplace=True) # rename these col so I can drop them.

In [9]:
# drop the rows that have one of the bewlow values in 'Acorn_grouped'
df = df.drop(df[(df.Acorn_grouped == "ACORN-U") | (df.Acorn_grouped == "ACORN-")].index)

In [10]:
df

,LCLid,stdorToU,DateTime,KWH/hh,Acorn,Acorn_grouped
0,MAC000002,Std,2012-10-12 00:30:00,0.000,ACORN-A,Affluent
1,MAC000002,Std,2012-10-12 01:00:00,0.000,ACORN-A,Affluent
2,MAC000002,Std,2012-10-12 01:30:00,0.000,ACORN-A,Affluent
3,MAC000002,Std,2012-10-12 02:00:00,0.000,ACORN-A,Affluent
4,MAC000002,Std,2012-10-12 02:30:00,0.000,ACORN-A,Affluent
...,...,...,...,...,...,...
167932469,MAC004221,Std,2013-08-06 04:30:00,0.017,ACORN-E,Affluent
167932470,MAC004221,Std,2013-08-06 05:00:00,0.017,ACORN-E,Affluent
167932471,MAC004221,Std,2013-08-06 05:30:00,0.017,ACORN-E,Affluent
167932472,MAC004221,Std,2013-08-06 06:00:00,0.017,ACORN-E,Affluent


In [11]:
gb1 = df.groupby('LCLid') # group by ID to be able to look at the start and end dates so as to figure out where to slice.

In [12]:
min(gb1.first().DateTime) # earlist start

Timestamp('2011-11-23 09:00:00')

In [13]:
 max(gb1.first().DateTime) # latest start

Timestamp('2013-10-29 18:23:02')

In [14]:
min(gb1.last().DateTime) # earlist stop

Timestamp('2012-05-01 21:00:00')

In [15]:
 max(gb1.last().DateTime) # latest stop

Timestamp('2014-02-28 00:00:00')

In [16]:
df2 = df.set_index(['DateTime'])
df2 = df2.loc['2013-01-01':'2014-01-01'] # 1 year of data.

In [17]:
df2

,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped
DateTime,,,,,
2013-01-01 00:00:00,MAC000002,Std,0.219,ACORN-A,Affluent
2013-01-01 00:30:00,MAC000002,Std,0.241,ACORN-A,Affluent
2013-01-01 01:00:00,MAC000002,Std,0.191,ACORN-A,Affluent
2013-01-01 01:30:00,MAC000002,Std,0.235,ACORN-A,Affluent
2013-01-01 02:00:00,MAC000002,Std,0.182,ACORN-A,Affluent
...,...,...,...,...,...
2013-08-06 04:30:00,MAC004221,Std,0.017,ACORN-E,Affluent
2013-08-06 05:00:00,MAC004221,Std,0.017,ACORN-E,Affluent
2013-08-06 05:30:00,MAC004221,Std,0.017,ACORN-E,Affluent


In [18]:
df2 = df2.reset_index() # Remove datetime from index.
gb2 = df2.groupby(['LCLid']).DateTime.count() # Group df by id and get count of the # of rows.
df3 = pd.DataFrame(gb2) # Put gb in df.
df3.rename(columns={'DateTime': '# of rows for each series'}, inplace=True)

In [19]:
df3

,# of rows for each series
LCLid,
MAC000002,17580
MAC000003,17578
MAC000004,17579
MAC000005,17579
MAC000006,17580
...,...
MAC005561,17575
MAC005562,17579
MAC005564,17571


In [20]:
lst_for_good_ids = [17580] # Series whos # of rows equals 17,580.
# Filter df for only series of len 17,580.
df4 = pd.DataFrame(df3.loc[df3['# of rows for each series'].isin(lst_for_good_ids)])

In [21]:
df6 = pd.DataFrame(df2.loc[df2['LCLid'].isin(df4.index.to_list())])
df6

,DateTime,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped
0,2013-01-01 00:00:00,MAC000002,Std,0.219,ACORN-A,Affluent
1,2013-01-01 00:30:00,MAC000002,Std,0.241,ACORN-A,Affluent
2,2013-01-01 01:00:00,MAC000002,Std,0.191,ACORN-A,Affluent
3,2013-01-01 01:30:00,MAC000002,Std,0.235,ACORN-A,Affluent
4,2013-01-01 02:00:00,MAC000002,Std,0.182,ACORN-A,Affluent
...,...,...,...,...,...,...
92477104,2014-01-01 21:30:00,MAC004219,Std,0.266,ACORN-E,Affluent
92477105,2014-01-01 22:00:00,MAC004219,Std,0.260,ACORN-E,Affluent
92477106,2014-01-01 22:30:00,MAC004219,Std,0.301,ACORN-E,Affluent
92477107,2014-01-01 23:00:00,MAC004219,Std,0.223,ACORN-E,Affluent


In [22]:
df6['Day'] = df6['DateTime'].dt.day_name() # Add the day of the week.

In [23]:
df_temp = pd.read_csv("data\\london 2013 temp.csv") # Load hourly temp data for London, UK in 2013.

In [24]:
df_temp
df_temp['DateTime'] = pd.to_datetime(df_temp['ob_time']) # convert to datetime datetype
df_temp.drop(['ob_time'], axis=1, inplace = True)

In [25]:
df7 = pd.merge(df6,df_temp ,on='DateTime',how='left')
df7['air_temperature'] = df7['air_temperature'].interpolate() # Fill in missing values for temp.
df7['air_temperature'] = df7['air_temperature'] * 1.8 + 32 # Convert to F degress.

In [26]:
df7

,DateTime,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped,Day,air_temperature
0,2013-01-01 00:00:00,MAC000002,Std,0.219,ACORN-A,Affluent,Tuesday,48.02
1,2013-01-01 00:30:00,MAC000002,Std,0.241,ACORN-A,Affluent,Tuesday,47.84
2,2013-01-01 01:00:00,MAC000002,Std,0.191,ACORN-A,Affluent,Tuesday,47.66
3,2013-01-01 01:30:00,MAC000002,Std,0.235,ACORN-A,Affluent,Tuesday,47.39
4,2013-01-01 02:00:00,MAC000002,Std,0.182,ACORN-A,Affluent,Tuesday,47.12
...,...,...,...,...,...,...,...,...
19654435,2014-01-01 21:30:00,MAC004219,Std,0.266,ACORN-E,Affluent,Wednesday,42.80
19654436,2014-01-01 22:00:00,MAC004219,Std,0.260,ACORN-E,Affluent,Wednesday,42.80
19654437,2014-01-01 22:30:00,MAC004219,Std,0.301,ACORN-E,Affluent,Wednesday,42.80
19654438,2014-01-01 23:00:00,MAC004219,Std,0.223,ACORN-E,Affluent,Wednesday,42.80


In [28]:
df7['Month'] = df7['DateTime'].dt.strftime('%B')

In [29]:
df7

,DateTime,LCLid,stdorToU,KWH/hh,Acorn,Acorn_grouped,Day,air_temperature,Month
0,2013-01-01 00:00:00,MAC000002,Std,0.219,ACORN-A,Affluent,Tuesday,48.02,January
1,2013-01-01 00:30:00,MAC000002,Std,0.241,ACORN-A,Affluent,Tuesday,47.84,January
2,2013-01-01 01:00:00,MAC000002,Std,0.191,ACORN-A,Affluent,Tuesday,47.66,January
3,2013-01-01 01:30:00,MAC000002,Std,0.235,ACORN-A,Affluent,Tuesday,47.39,January
4,2013-01-01 02:00:00,MAC000002,Std,0.182,ACORN-A,Affluent,Tuesday,47.12,January
...,...,...,...,...,...,...,...,...,...
19654435,2014-01-01 21:30:00,MAC004219,Std,0.266,ACORN-E,Affluent,Wednesday,42.80,January
19654436,2014-01-01 22:00:00,MAC004219,Std,0.260,ACORN-E,Affluent,Wednesday,42.80,January
19654437,2014-01-01 22:30:00,MAC004219,Std,0.301,ACORN-E,Affluent,Wednesday,42.80,January
19654438,2014-01-01 23:00:00,MAC004219,Std,0.223,ACORN-E,Affluent,Wednesday,42.80,January


In [30]:
df7.to_csv("C:\\Users\\jesse\\Downloads\\clases\\electrical data\\london data\\data\\2013_01_01_2014_01_01_day_temp.csv", index = False) # full dataset